# Piperack Design Optimiser 1.1

## Module Imports and connection to STAAD

In [ ]:
from base.helper import *
from base.staad_base.geometry import *
from base.staad_base.load import *
from base.staad_base.design import *
from base.staad_base.property import *
from base.staad_base.optimise_member import *
from IPython.display import display, Markdown
from pyperclip import copy

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output

beam_objects:dict = get_beam_objects(geometry=geometry,property=property)
beam_nos:list = list(beam_objects.keys())

### STAAD Members

In [ ]:
markdown_output = ''
markdown_output = "### Beam Properties\n\n"
markdown_output += "| Beam ID | Start | End | Length | Profile |\n"
markdown_output += "| --- | --- | --- | --- | --- |\n"

beam_items = beam_objects.items()

beam:Beam3D
for beamNo,beam in beam_items:
    markdown_output += f"| {beamNo} | {beam.start} | {beam.end} | {beam.start.distance_to(beam.end)} | {beam.profile} | \n"

display(Markdown(markdown_output))

### Member Selection

In [ ]:
selected_members = get_selected_beam_nos(geometry)
copy(f'members={selected_members},')

display(Markdown(f'copied **{len(selected_members)}** members : {selected_members}'))

## Member Groups

In [ ]:
'''BEAMS'''
beams_tier_1 = member_group(
    id='tier_1',
    members=[],
    profiles=[],
    Staad_objects= STAAD_objects
)


optimise_group_collection = optimise_groups().add(beams_tier_1)


markdown_output = "# Optimise Group Profile Data \n\n"
markdown_output += "| Group ID | Current Profile | Check Profiles | Members |\n"
markdown_output += "|-----------|-------------------|-------------------|------------------|\n"

group_i:member_group
for group_i in optimise_group_collection.member_groups:
    members_str = ",".join(str(x) for x in group_i.members)
    markdown_output += f"| {group_i.id} | {group_i.profile_name} | {','.join(str(x) for x in group_i.profiles)} | <div style='width: 600px; overflow-wrap: break-word;'>{members_str}</div> |\n"

display(Markdown(markdown_output))

## Optimiser Function

In [ ]:
markdown_output = "# Optimization Results\n\n"

for group in optimise_group_collection.member_groups:
    print(f'\n\nOptimising member group {group.id} for {len(group.profiles)} profiles : {group.profiles}')
    
    markdown_output += f"## Optimising Member Group {group.id}\n"
    markdown_output += f"- **Number of Profiles**: {len(group.profiles)}\n"
    markdown_output += f"- **Profiles**: {group.profiles}\n\n"

    print('Setting default profile for all groups')
    markdown_output += "### Setting Default Profile for All Groups\n\n"
    for group_i in optimise_group_collection.member_groups:
        group_i.set_members_property_initial()

    for index in range(len(group.profiles)):
        print(f'\nProfile id : {group.profiles[index]}')
        markdown_output += f"#### Profile ID: {group.profiles[index]}\n"

        group.set_members_property(index)
        run_analysis(openSTAAD)
        group.get_utilization_ratios_for_profile(index)
        
        markdown_output += "# Group Result\n\n"
        markdown_output += "| Profile | Number of Failed Members | Failed Members | Average Utilization Ratio | Standard Deviation |\n"
        markdown_output += "|---------|-------------------------|---------------|---------------------------|--------------------|\n"

        # Assuming group.results[index] contains the data
        profile = group.results[index]['profile']
        num_failed = group.results[index]['failed']
        failed_members = [x for (x, y) in group.results[index]['failed_members']] or 'None'
        failed_members_str = ",".join(str(x) for x in failed_members) if failed_members != 'None' else 'None'
        average = group.results[index]['average']
        deviation = group.results[index]['deviation']

        # Wrap Failed Members in a div with fixed width and text wrapping
        failed_members_formatted = f"<div style='width: 100px; overflow-wrap: break-word;'>{failed_members_str}</div>"
        markdown_output += f"| {profile} | {num_failed} | {failed_members_formatted} | {average:.4f} | {deviation:.4f} |\n"

        if group.results[index]['failed_members']:
            markdown_output += "\n**Failed Utilization Ratios**\n"
            markdown_output += "| Member ID | Utilization Ratio |\n"
            markdown_output += "|-----------|-------------------|\n"
            for member_id, ratio in group.results[index]['failed_members']:
                markdown_output += f"| {member_id} | {ratio:.4f} |\n"

        all_members = member_group(
            id='whole_structure',
            members=beam_nos,
            exclude_members=group.members,
            Staad_objects=STAAD_objects,
        )
        all_members.get_utilization_ratios()
        
        markdown_output += "# Model Result (Whole Structure)\n\n"
        markdown_output += "| Number of Failed Members | Failed Members | Average Utilization Ratio | Standard Deviation |\n"
        markdown_output += "|-------------------------|---------------|---------------------------|--------------------|\n"

        # Extract data from all_members.results
        num_failed = all_members.results['failed']
        failed_members = [x for (x, y) in all_members.results['failed_members']] or 'None'
        failed_members_str = ",".join(str(x) for x in failed_members) if failed_members != 'None' else 'None'
        average = all_members.results['average']
        deviation = all_members.results['deviation']

        # Wrap Failed Members in a div with fixed width and text wrapping
        failed_members_formatted = f"<div style='width: 100px; overflow-wrap: break-word;'>{failed_members_str}</div>"
        markdown_output += f"| {num_failed} | {failed_members_formatted} | {average:.4f} | {deviation:.4f} |\n"

        model_ratios = {k: v for k, v in all_members.results['result'].items() if v != 0.0}
        if all_members.results['failed_members']:
            markdown_output += "\n**Failed Utilization Ratios**\n"
            markdown_output += "| Member ID | Utilization Ratio |\n"
            markdown_output += "|-----------|-------------------|\n"
            for member_id, ratio in all_members.results['failed_members']:
                markdown_output += f"| {member_id} | {ratio:.4f} |\n"

display(Markdown(markdown_output))

### Reset Profiles (if optimiser interuptted)

In [ ]:
markdown_output = "# Group Profile Data \n\n"
markdown_output += "| Group ID | Profile |\n"
markdown_output += "|-----------|-------------------|\n"

group_i:member_group
for group_i in optimise_group_collection.member_groups:
    markdown_output += f"| {group_i.id} | {group_i.profile_name} |\n"

display(Markdown(markdown_output))

permission = input("Assign profiles ? (y/n)")

if(permission == 'y'):
    for group_i in optimise_group_collection.member_groups:
        group_i.set_members_property_initial()


### Loop through groups

In [ ]:
group_i : member_group
for group_i in optimise_group_collection.member_groups:
    group_i.select_members()
    display(Markdown(f'Selected {group_i.id}'))
    time.sleep(5)

## Final Analysis

In [ ]:
# run_analysis(openSTAAD)

all_members = member_group(
            id='whole_structure',
            members=beam_nos,
            Staad_objects=STAAD_objects,
            allowable_ratio=0.85
        )
all_members.get_utilization_ratios()

markdown_output = "# Model Result (Whole Structure)\n\n"
markdown_output += "| Number of Failed Members | Failed Members | Average Utilization Ratio | Standard Deviation |\n"
markdown_output += "|-------------------------|---------------|---------------------------|--------------------|\n"

# Extract data from all_members.results
num_failed = all_members.results['failed']
failed_members = [x for (x, y) in all_members.results['failed_members']] or 'None'
failed_members_str = ",".join(str(x) for x in failed_members) if failed_members != 'None' else 'None'
average = all_members.results['average']
deviation = all_members.results['deviation']

# Wrap Failed Members in a div with fixed width and text wrapping
failed_members_formatted = f"<div style='width: 250px; overflow-wrap: break-word;'>{failed_members_str}</div>"
markdown_output += f"| {num_failed} | {failed_members_formatted} | {average:.4f} | {deviation:.4f} |\n"

model_ratios = {k: v for k, v in all_members.results['result'].items() if v != 0.0}
if all_members.results['failed_members']:
    markdown_output += "\n**Failed Utilization Ratios**\n"
    markdown_output += "| Member ID | Utilization Ratio |\n"
    markdown_output += "|-----------|-------------------|\n"
    for member_id, ratio in all_members.results['failed_members']:
        markdown_output += f"| {member_id} | {ratio:.4f} |\n"

display(Markdown(markdown_output))